# Tinder para empresas

## Imports

In [1]:
# Imports

import numpy as np
import pandas as pd

from random import shuffle

from func_matching import calculate_match
from func_args import get_args
from func_grafos import create_graph, get_best_match
from func_aquisition import adquisition_survey
from func_outputs import *
from func_normalize import *

In [2]:

def most_common(lst):
    return max(set(lst), key=lst.count)

def get_rondas(G, n_rondas, student_queue, companies): 
    res = []
    mesas = len(companies)
    for r in range(n_rondas): 
        used = []
        
        for interview in range(mesas): 
            # debería ser el studiante que más sale en student_queue
            student = most_common(student_queue) 
            
            m = get_best_match(G, student, used)
            if m == None: 
                break
            node1, node2, w = m
            G.remove_edge(node1, node2)
            used.extend([node1, node2]) # añadimos empresa y compañia para que no vuelvan a aparecer en esta ronda
            
            if node1 not in companies: # si el nodo1 no es nombre de compañía, debemos intercambiar
                node2, node1 = node1, node2
            res.append({'ronda': r, 'company': node1, 'student': node2, 'weight': w})
            student_queue.remove(node2)
    return res    

def generate_student_queue(students_list, n_rondas): 
    shuffle(students_list)
    return students_list * n_rondas

def report_matching(df): 
    '''
    Informe de mínimo de entrevistas, máximo de entrevistas, 
    rondas, etc...
    '''
    pass

def outputs(rondas): 
    df = pd.DataFrame.from_dict(rondas, orient='columns')
    display(df)
    
    df_matrix = create_matrix_ronda_empresa(rondas)
    display(df_matrix)
    
    df_matrix2 = create_matrix_ronda_estudiante(rondas)
    display(df_matrix2)
    
    report_matching(df)
    
    writer = pd.ExcelWriter('output/output.xlsx')
    df.to_excel(writer,'Sheet1')
    df_matrix.to_excel(writer,'Sheet2')
    df_matrix2.to_excel(writer,'Sheet3')
    writer.save()
    
    # check
    '''
    for r in range(11): 
        ronda10a = set(df[df['ronda']==r]['student']) 
        ronda10b = set(df_matrix.iloc[r, :])
        display(ronda10a == ronda10b)
    '''

In [3]:
def main(): 
    # Toma de argumentos. 
    bootcamp, companies_filename, students_filename, n_rondas = get_args()
    
    # DFs limpios
    companies, students = adquisition_survey(bootcamp, companies_filename, students_filename)
    
    # Normalize
    students, companies = normalize_2dfs(students, companies)
    
    # TO DO: crear la tabla matching
    matching = calculate_match(students, companies, bootcamp)

    G = create_graph(matching)
    
    students_list_queue = generate_student_queue(list(matching.index), n_rondas)
    # display(students_list_queue)
        
    # vamos a recorrer la lista para ir emparejando
    rondas = get_rondas(G, n_rondas, students_list_queue, list(matching.columns))
    #display(rondas)
    
    outputs(rondas)


if __name__== '__main__':
    main()

,company,ronda,student,weight
0,InboundCycle,0,Sergio Ortega Gil,1.000000
1,Bit Genoma Digital Solutions SL,0,Maria Isabel Llanes Agudelo,0.381652
2,AKKA,0,Enrique Anaya Marin,0.385394
3,Project Lobster,0,Francisco Leston Ramos,0.348331
4,Atomian,0,Verónica Celemín Lominchar,0.256808
5,Bit Genoma Digital Solutions,0,Javier Pérez-Chao Rodríguez,0.387426
6,C14TORCE,0,Daniel García Martínez,0.225844
7,The Hotels Network,0,Mario Quiroga Cuervo,0.238007
8,SEAT,0,Isabel Rodríguez Godoy,0.209428
9,ONEBOX,0,Abel Colmenares Napolitano,0.198763


,InboundCycle,Bit Genoma Digital Solutions SL,AKKA,Project Lobster,Atomian,Bit Genoma Digital Solutions,C14TORCE,The Hotels Network,SEAT,ONEBOX,Norwegian,Volcanic Internet,Farmaconnect,Visyon (Estudio Bay SL),Bluekik,Exoticca,Movetia,CAPITOLE CONSULTING
0,Sergio Ortega Gil,Maria Isabel Llanes Agudelo,Enrique Anaya Marin,Francisco Leston Ramos,Verónica Celemín Lominchar,Javier Pérez-Chao Rodríguez,Daniel García Martínez,Mario Quiroga Cuervo,Isabel Rodríguez Godoy,Abel Colmenares Napolitano,Daniel Herrero Hernando,Alfonso Silva Feria,Thilo Sperber,Sofia Jordan,Baltasar Torres Garcia,Alfredo Hinarejos Corbeá,Patricia Defez Díaz de Sarralde,Maria Cristina Blanco
1,Dan Mihai Grigoriciuc,Eugenio Ruiz Cepeda,Javier Requejo Melchor,Nira Alonso Rodríguez,Jaime Jacobo Rodríguez del Valle,Carlos Eduardo Lores Hernández,Nerea Aparicio,Moisés García Almeida,Maria Isabel Llanes Agudelo,Roberto Muñoz Aragón,Enrique Anaya Marin,Carlos Martínez Crespo,Rodrigo Otero Montero,Sergio Ortega Gil,Francisco Leston Ramos,Verónica Celemín Lominchar,Javier Pérez-Chao Rodríguez,Daniel García Martínez
2,Isabel Rodríguez Godoy,Mario Quiroga Cuervo,Abel Colmenares Napolitano,Alfonso Silva Feria,Thilo Sperber,Sofia Jordan,Daniel Herrero Hernando,Baltasar Torres Garcia,Maria Cristina Blanco,Alfredo Hinarejos Corbeá,Dan Mihai Grigoriciuc,Patricia Defez Díaz de Sarralde,Eugenio Ruiz Cepeda,Javier Requejo Melchor,Nerea Aparicio,Nira Alonso Rodríguez,Carlos Eduardo Lores Hernández,Jaime Jacobo Rodríguez del Valle
3,Carlos Martínez Crespo,Moisés García Almeida,Roberto Muñoz Aragón,Rodrigo Otero Montero,Sergio Ortega Gil,Enrique Anaya Marin,Maria Isabel Llanes Agudelo,Daniel García Martínez,Javier Pérez-Chao Rodríguez,Verónica Celemín Lominchar,Abel Colmenares Napolitano,Francisco Leston Ramos,Isabel Rodríguez Godoy,Mario Quiroga Cuervo,Alfonso Silva Feria,Daniel Herrero Hernando,Thilo Sperber,Sofia Jordan
4,Patricia Defez Díaz de Sarralde,Alfredo Hinarejos Corbeá,Baltasar Torres Garcia,Maria Cristina Blanco,Dan Mihai Grigoriciuc,Nira Alonso Rodríguez,Eugenio Ruiz Cepeda,Nerea Aparicio,Carlos Eduardo Lores Hernández,Javier Requejo Melchor,Rodrigo Otero Montero,Roberto Muñoz Aragón,Jaime Jacobo Rodríguez del Valle,Carlos Martínez Crespo,Moisés García Almeida,Sergio Ortega Gil,Maria Isabel Llanes Agudelo,Enrique Anaya Marin
5,Francisco Leston Ramos,Javier Pérez-Chao Rodríguez,Verónica Celemín Lominchar,Daniel García Martínez,Abel Colmenares Napolitano,Daniel Herrero Hernando,Mario Quiroga Cuervo,Isabel Rodríguez Godoy,Sofia Jordan,Thilo Sperber,Patricia Defez Díaz de Sarralde,Baltasar Torres Garcia,Alfonso Silva Feria,Alfredo Hinarejos Corbeá,Eugenio Ruiz Cepeda,Maria Cristina Blanco,Dan Mihai Grigoriciuc,Javier Requejo Melchor
6,Carlos Eduardo Lores Hernández,Nerea Aparicio,Nira Alonso Rodríguez,Jaime Jacobo Rodríguez del Valle,Rodrigo Otero Montero,Carlos Martínez Crespo,Roberto Muñoz Aragón,Maria Isabel Llanes Agudelo,Moisés García Almeida,Enrique Anaya Marin,Verónica Celemín Lominchar,Sergio Ortega Gil,Francisco Leston Ramos,Javier Pérez-Chao Rodríguez,Daniel García Martínez,Mario Quiroga Cuervo,Isabel Rodríguez Godoy,Abel Colmenares Napolitano
7,Alfonso Silva Feria,Daniel Herrero Hernando,Thilo Sperber,Sofia Jordan,Alfredo Hinarejos Corbeá,Patricia Defez Díaz de Sarralde,Baltasar Torres Garcia,Maria Cristina Blanco,Eugenio Ruiz Cepeda,Nira Alonso Rodríguez,Carlos Eduardo Lores Hernández,Dan Mihai Grigoriciuc,Javier Requejo Melchor,Nerea Aparicio,Roberto Muñoz Aragón,Jaime Jacobo Rodríguez del Valle,Rodrigo Otero Montero,Carlos Martínez Crespo
8,Maria Isabel Llanes Agudelo,Enrique Anaya Marin,Sergio Ortega Gil,Moisés García Almeida,Francisco Leston Ramos,Daniel García Martínez,Verónica Celemín Lominchar,Javier Pérez-Chao Rodríguez,Mario Quiroga Cuervo,Daniel Herrero Hernando,Thilo Sperber,Isabel Rodríguez Godoy,Abel Colmenares Napolitano,Alfonso Silva Feria,Alfredo Hinarejos Corbeá,Sofia Jordan,Baltasar Torres Garcia,Patricia Defez Díaz de Sarralde
9,Eugenio Ruiz

,Sergio Ortega Gil,Maria Isabel Llanes Agudelo,Enrique Anaya Marin,Francisco Leston Ramos,Verónica Celemín Lominchar,Javier Pérez-Chao Rodríguez,Daniel García Martínez,Mario Quiroga Cuervo,Isabel Rodríguez Godoy,Abel Colmenares Napolitano,...,Dan Mihai Grigoriciuc,Javier Requejo Melchor,Nira Alonso Rodríguez,Nerea Aparicio,Carlos Eduardo Lores Hernández,Jaime Jacobo Rodríguez del Valle,Roberto Muñoz Aragón,Rodrigo Otero Montero,Carlos Martínez Crespo,Moisés García Almeida
0,InboundCycle,Bit Genoma Digital Solutions SL,AKKA,Project Lobster,Atomian,Bit Genoma Digital Solutions,C14TORCE,The Hotels Network,SEAT,ONEBOX,...,,,,,,,,,,
1,Visyon (Estudio Bay SL),SEAT,Norwegian,Bluekik,Exoticca,Movetia,CAPITOLE CONSULTING,,,,...,InboundCycle,AKKA,Project Lobster,C14TORCE,Bit Genoma Digital Solutions,Atomian,ONEBOX,Farmaconnect,Volcanic Internet,The Hotels Network
2,,,,,,,,Bit Genoma Digital Solutions SL,InboundCycle,AKKA,...,Norwegian,Visyon (Estudio Bay SL),Exoticca,Bluekik,Movetia,CAPITOLE CONSULTING,,,,
3,Atomian,C14TORCE,Bit Genoma Digital Solutions,Volcanic Internet,ONEBOX,SEAT,The Hotels Network,Visyon (Estudio Bay SL),Farmaconnect,Norwegian,...,,,,,,,AKKA,Project Lobster,InboundCycle,Bit Genoma Digital Solutions SL
4,Exoticca,Movetia,CAPITOLE CONSULTING,,,,,,,,...,Atomian,ONEBOX,Bit Genoma Digital Solutions,The Hotels Network,SEAT,Farmaconnect,Volcanic Internet,Norwegian,Visyon (Estudio Bay SL),Bluekik
5,,,,InboundCycle,AKKA,Bit Genoma Digital Solutions SL,Project Lobster,C14TORCE,The Hotels Network,Atomian,...,Movetia,CAPITOLE CONSULTING,,,,,,,,
6,Volcanic Internet,The Hotels Network,ONEBOX,Farmaconnect,Norwegian,Visyon (Estudio Bay SL),Bluekik,Exoticca,Movetia,CAPITOLE CONSULTING,...,,,AKKA,Bit Genoma Digital Solutions SL,InboundCycle,Project Lobster,C14TORCE,Atomian,Bit Genoma Digital Solutions,SEAT
7,,,,,,,,,,,...,Volcanic Internet,Farmaconnect,ONEBOX,Visyon (Estudio Bay SL),Norwegian,Exoticca,Bluekik,Movetia,CAPITOLE CONSULTING,
8,AKKA,InboundCycle,Bit Genoma Digital Solutions SL,Atomian,C14TORCE,The Hotels Network,Bit Genoma Digital Solutions,SEAT,Volcanic Internet,Farmaconnect,...,,,,,,,,,,Project Lobster
9,Norwegian,ONEBOX,Exoticca,Movetia,Bluekik,CAPITOLE CONSULTING,,,,,...,Bit Genoma Digital Solutions SL,Project Lobster,C14TORCE,Bit Genoma Digital Solutions,Atomian,Volcanic Internet,The Hotels Network,SEAT,Farmaconnect,Visyon (Estudio Bay SL)
